In [ ]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import logging
import shutil

# Module importieren
from utils.logging_setup import setup_logging
from data.data_loader import load_price_data
from models.energy_storage import EnergyStorage
from strategies.threshold_lookahead import threshold_lookahead
from visualization.parallel_plotting import create_visualizations_parallel
from economic_analysis.analyzer import run_economic_analysis
from visualization.plotting import (
    plot_cycle_analysis,
    plot_charge_discharge_patterns,
    plot_efficiency_analysis,
    plot_price_arbitrage_analysis,
)
import config


# --- Globale Stil-Einstellungen für alle Diagramme (für die Bachelorarbeit optimiert) ---
plt.rcParams.update({
    'font.size': 14,                     # Allgemeine Schriftgröße
    'axes.titlesize': 18,                # Titel der Diagramme
    'axes.labelsize': 16,                # Beschriftung der X- und Y-Achse
    'xtick.labelsize': 12,               # Schriftgröße der X-Achsen-Werte
    'ytick.labelsize': 12,               # Schriftgröße der Y-Achsen-Werte
    'legend.fontsize': 12,               # Schriftgröße der Legende
    'figure.titlesize': 20,              # Haupt-Titel der Abbildung (falls verwendet)
    'lines.linewidth': 2.5,              # Dicke der Linien in Liniendiagrammen
    'lines.markersize': 8                # Größe von Markern in Plots
})


def run_continuous_simulation_for_window(prices_df, window_size, output_dir, save_plots=True, show_plots=False):
    """
    Führt eine kontinuierliche Simulation für eine spezifische Window Size durch.
    Erstellt alle Visualisierungen wie in der originalen main.py.
    
    Returns:
    --------
    tuple: (daily_results, storage, annual_profit, annual_cycles)
    """
    logger = logging.getLogger(__name__)
    logger.info(f"\n=== Starte kontinuierliche Simulation mit Window Size {window_size} ===")
    
    # Initialisiere Speicher
    storage = EnergyStorage(
        capacity_mwh=config.CAPACITY_MWH,
        charge_rate=config.CHARGE_DISCHARGE_RATE,
        efficiency=config.EFFICIENCY,
        fee_per_mwh=config.FEE_PER_MWH
    )
    
    # Threshold-Lookahead-Strategie anwenden
    start_time = time.time()
    storage, energy_history = threshold_lookahead(
        prices_df, 
        storage, 
        window_size=window_size
    )
    elapsed_time = time.time() - start_time
    logger.info(f"Optimierung abgeschlossen in {elapsed_time:.2f} Sekunden")
    
    # Zusammenfassung der globalen Simulation
    total_transactions = len(storage.transactions)
    total_charged = sum([t.get('amount', 0) for t in storage.transactions if t['type'] == 'charge'])
    total_discharged = sum([t.get('amount_gross', t.get('amount', 0)) for t in storage.transactions if t['type'] == 'discharge'])
    total_profit = storage.cash
    
    logger.info(f"Gesamtzahl Transaktionen: {total_transactions}")
    logger.info(f"Gesamtgewinn: {total_profit:.2f} €")
    logger.info(f"Gesamtzyklen: {storage.total_cycles:.2f}")
    
    # Verzeichnis für tägliche Visualisierungen
    daily_viz_dir = os.path.join(output_dir, "handelsentscheidungen")
    os.makedirs(daily_viz_dir, exist_ok=True)
    
    # Vorverarbeitung der Daten nach Datum
    logger.info("Gruppiere Transaktionen und Energiehistorie nach Datum...")
    
    # Transaktionen nach Datum gruppieren
    transactions_by_date = {}
    for t in storage.transactions:
        if t['index'] < len(prices_df):
            tx_time = prices_df.iloc[t['index']]['datetime']
            date = tx_time.date()
            if date not in transactions_by_date:
                transactions_by_date[date] = []
            transactions_by_date[date].append(t.copy())
    
    # Energiehistorie nach Datum gruppieren
    energy_by_date = {}
    for eh in energy_history:
        if eh['time_index'] < len(prices_df):
            eh_time = prices_df.iloc[eh['time_index']]['datetime']
            date = eh_time.date()
            if date not in energy_by_date:
                energy_by_date[date] = []
            energy_by_date[date].append(eh.copy())
    
    # Mapping zwischen globalen und täglichen Indizes erstellen
    index_mapping_by_date = {}
    for date, date_dt in ((d, pd.to_datetime(d)) for d in sorted(set(prices_df['datetime'].dt.date))):
        day_prices = prices_df[prices_df['datetime'].dt.date == date].reset_index()
        
        global_to_day_index = {}
        for i, row in day_prices.iterrows():
            if 'index' in row:
                global_to_day_index[row['index']] = i
        
        index_mapping_by_date[date] = global_to_day_index
    
    # Ergebnisse pro Tag auswerten
    all_dates = sorted(prices_df['datetime'].dt.date.unique())
    daily_results = []
    
    logger.info("Erstelle tägliche Ergebnisse...")
    
    for day_index, date in enumerate(all_dates):
        if day_index % 50 == 0:
            logger.info(f"Verarbeite Tag {day_index+1}/{len(all_dates)}")
        
        date_dt = pd.to_datetime(date)
        
        # Direkter Zugriff auf die vorgruppierte Daten
        day_transactions = transactions_by_date.get(date, [])
        day_transactions_df = pd.DataFrame(day_transactions) if day_transactions else pd.DataFrame()
        
        # Filtern der Preisdaten für diesen Tag
        day_prices = prices_df[prices_df['datetime'].dt.date == date].copy().reset_index()
        
        # Verwende das vorberechnete Mapping
        global_to_day_index = index_mapping_by_date.get(date, {})
        
        # Transaktionen-Indizes anpassen
        if not day_transactions_df.empty and 'index' in day_transactions_df.columns:
            day_transactions_df['original_index'] = day_transactions_df['index'].copy()
            day_transactions_df['index'] = day_transactions_df['index'].apply(
                lambda x: global_to_day_index.get(x, 0) if x in global_to_day_index else 0
            )
        
        # Direkter Zugriff auf die vorgruppierte Energiehistorie
        day_energy_history = energy_by_date.get(date, [])
        
        # Tagesenergie-Indizes anpassen
        for eh in day_energy_history:
            if eh['time_index'] in global_to_day_index:
                eh['time_index'] = global_to_day_index[eh['time_index']]
            else:
                eh['time_index'] = 0
        
        # Tägliche Statistiken berechnen
        if not day_transactions_df.empty:
            # Berechne Tagesgewinn
            day_profit = 0
            for t in day_transactions:
                if t['type'] == 'discharge':
                    day_profit += t.get('revenue', 0)
                else:
                    day_profit -= t.get('cost', 0)
            
            # Berechne Tageszyklen
            day_discharge = sum([t.get('amount_gross', t.get('amount', 0)) 
                               for t in day_transactions if t['type'] == 'discharge'])
            day_cycles = day_discharge / config.CAPACITY_MWH
            
            # Bestimme Anfangs- und Endladestand
            if day_energy_history:
                initial_energy = day_energy_history[0]['energy_level']
                final_energy = day_energy_history[-1]['energy_level']
            else:
                initial_energy = 0
                final_energy = 0
            
            # Anzahl der Handelsgeschäfte
            charge_count = len([t for t in day_transactions if t['type'] == 'charge'])
            discharge_count = len([t for t in day_transactions if t['type'] == 'discharge'])
            
            # Tagesergebnis speichern
            daily_result = {
                'date': date_dt,
                'profit': day_profit,
                'cycles_completed': day_cycles,
                'charge_count': charge_count,
                'discharge_count': discharge_count,
                'initial_energy_level': initial_energy,
                'final_energy_level': final_energy,
                'transactions': day_transactions_df,
                'prices_df': day_prices,
                'energy_history': day_energy_history
            }
            
            # Erweiterte Statistiken
            day_charged = sum([t.get('amount', 0) for t in day_transactions if t['type'] == 'charge'])
            day_discharged_gross = sum([t.get('amount_gross', t.get('amount', 0)) 
                                     for t in day_transactions if t['type'] == 'discharge'])
            day_discharged_net = sum([t.get('amount_usable', t.get('amount', 0) * config.EFFICIENCY) 
                                   for t in day_transactions if t['type'] == 'discharge'])
            
            daily_result.update({
                'total_charged': day_charged,
                'total_gross_energy': day_discharged_gross,
                'total_usable_energy': day_discharged_net,
                'total_energy_loss': day_discharged_gross - day_discharged_net,
                'total_fees': sum([t.get('transaction_fee', 0) for t in day_transactions]),
                'efficiency_losses': sum([t.get('efficiency_loss', 0) for t in day_transactions if t['type'] == 'discharge'])
            })
            
            daily_results.append(daily_result)
        else:
            # Leeres Tagesergebnis
            daily_results.append({
                'date': date_dt,
                'profit': 0,
                'cycles_completed': 0,
                'charge_count': 0,
                'discharge_count': 0,
                'initial_energy_level': 0 if not day_energy_history else day_energy_history[0]['energy_level'],
                'final_energy_level': 0 if not day_energy_history else day_energy_history[-1]['energy_level'],
                'transactions': pd.DataFrame(),
                'prices_df': day_prices,
                'energy_history': day_energy_history,
                'total_charged': 0,
                'total_gross_energy': 0,
                'total_usable_energy': 0,
                'total_energy_loss': 0,
                'total_fees': 0,
                'efficiency_losses': 0
            })
    
    # Berechnung für jährliche Hochrechnung
    days_in_data = len(all_dates)
    total_profit = sum(result['profit'] for result in daily_results)
    total_cycles = sum(result['cycles_completed'] for result in daily_results)
    
    annual_profit = total_profit * (365 / days_in_data) if days_in_data < 365 else total_profit
    annual_cycles = total_cycles * (365 / days_in_data) if days_in_data < 365 else total_cycles
    
    logger.info(f"Gesamtgewinn aus {days_in_data} Tagen: {total_profit:.2f} €")
    logger.info(f"Gesamtzyklen: {total_cycles:.2f}")
    
    return daily_results, storage, annual_profit, annual_cycles


def test_window_sizes_with_full_visualization():
    """
    Testet verschiedene Fenstergrößen und erstellt für jede die vollständigen Visualisierungen.
    """
    # Logger setup
    logger = setup_logging()
    logger.info("Starte erweiterte Analyse verschiedener Window Sizes mit vollständigen Visualisierungen")
    
    # Basisverzeichnis für die Ausgabe
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_output_dir = os.path.join(config.OUTPUT_DIR, f"window_analysis_full_{timestamp}")
    os.makedirs(base_output_dir, exist_ok=True)
    
    # Preisdaten laden
    prices_df = load_price_data(config.DEFAULT_PRICE_FILE)
    if prices_df is None:
        logger.error("Fehler beim Laden der Daten. Bitte Dateipfad überprüfen.")
        return
    
    # Definition der zu testenden Window Sizes
    window_sizes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                    16, 17, 18, 19, 20]
    
    # Ergebnisse speichern
    results = []
    
    # Startzeit für Gesamtlaufzeit
    total_start_time = time.time()
    
    # Jede Window Size testen
    for window_size in window_sizes:
        logger.info(f"\n{'='*80}")
        logger.info(f"TESTE WINDOW SIZE: {window_size} (entspricht {window_size*15} Minuten)")
        logger.info(f"{'='*80}")
        
        # Spezifischen Ausgabeordner für diese Window Size erstellen
        window_dir = os.path.join(base_output_dir, f"window_{window_size:02d}")
        os.makedirs(window_dir, exist_ok=True)
        
        # Startzeit für diese Simulation
        start_time = time.time()
        
        try:
            # Kontinuierliche Simulation mit vollständiger Analyse durchführen
            daily_results, storage, annual_profit, annual_cycles = run_continuous_simulation_for_window(
                prices_df, 
                window_size,
                output_dir=window_dir,
                save_plots=True,
                show_plots=False
            )
            
            # Phase 2: Parallele Tagesvisualisierungen erstellen
            logger.info(f"Erstelle Tagesvisualisierungen für Window Size {window_size}...")
            daily_viz_dir = os.path.join(window_dir, "handelsentscheidungen")
            
            # Nur jeden n-ten Tag visualisieren für Effizienz
            visualization_frequency = 10  # Jeden 10. Tag
            
            create_visualizations_parallel(
                daily_results,
                prices_df,
                output_dir=daily_viz_dir,
                save_plots=True,
                show_plots=False,
                visualization_frequency=visualization_frequency
            )
            
            # Wirtschaftliche Analyse
            logger.info(f"Führe wirtschaftliche Analyse für Window Size {window_size} durch...")
            try:
                economic_results = run_economic_analysis(
                    prices_df, 
                    output_dir=window_dir,
                    annual_profit=annual_profit,
                    annual_cycles=annual_cycles,
                    daily_results=daily_results
                )
            except Exception as e:
                logger.warning(f"Fehler bei der wirtschaftlichen Analyse: {e}")
                economic_results = None
            
            # Batterie-Parameter-Visualisierungen
            logger.info(f"Erstelle Batterie-Parameter-Visualisierungen für Window Size {window_size}...")
            battery_dir = os.path.join(window_dir, "batterie_parameter")
            os.makedirs(battery_dir, exist_ok=True)
            battery_path = os.path.join(battery_dir, "battery_analysis")
            
            try:
                # Zyklenanalyse
                cycle_results = plot_cycle_analysis(daily_results, battery_path)
                logger.info(f"Zyklenanalyse erstellt: Gesamt {cycle_results.get('total_cycles', 0):.1f} Zyklen")
            except Exception as e:
                logger.warning(f"Fehler bei der Zyklenanalyse: {str(e)}")
            
            try:
                # Lade-/Entlademuster
                pattern_results = plot_charge_discharge_patterns(daily_results, battery_path)
                logger.info(f"Lade-/Entlademuster-Analyse erstellt")
            except Exception as e:
                logger.warning(f"Fehler bei der Lade-/Entlademuster-Analyse: {str(e)}")
            
            try:
                # Effizienzanalyse
                efficiency_results = plot_efficiency_analysis(daily_results, battery_path)
                logger.info(f"Effizienzanalyse erstellt: Ø Kapazitätsnutzung {efficiency_results.get('avg_capacity_utilization', 0):.2f}%")
            except Exception as e:
                logger.warning(f"Fehler bei der Effizienzanalyse: {str(e)}")
            
            try:
                # Preis-Arbitrage
                arbitrage_results = plot_price_arbitrage_analysis(daily_results, battery_path)
                logger.info(f"Preis-Arbitrage-Analyse erstellt: Ø Handelsmarge {arbitrage_results.get('avg_margin', 0):.2f} €/MWh")
            except Exception as e:
                logger.warning(f"Fehler bei der Preis-Arbitrage-Analyse: {str(e)}")
            
            # Statistiken berechnen
            total_transactions = len(storage.transactions)
            if total_transactions > 0:
                charge_prices = [t['price'] for t in storage.transactions if t['type'] == 'charge']
                discharge_prices = [t['price'] for t in storage.transactions if t['type'] == 'discharge']
                
                avg_charge_price = np.mean(charge_prices) if charge_prices else 0
                avg_discharge_price = np.mean(discharge_prices) if discharge_prices else 0
                price_spread = avg_discharge_price - avg_charge_price
            else:
                avg_charge_price = 0
                avg_discharge_price = 0
                price_spread = 0
            
            # Ergebnis für diese Window Size speichern
            result = {
                'window_size': window_size,
                'window_minutes': window_size * 15,
                'window_hours': window_size / 4,
                'profit': annual_profit,
                'transactions': total_transactions,
                'charge_transactions': len([t for t in storage.transactions if t['type'] == 'charge']),
                'discharge_transactions': len([t for t in storage.transactions if t['type'] == 'discharge']),
                'cycles': annual_cycles,
                'avg_charge_price': avg_charge_price,
                'avg_discharge_price': avg_discharge_price,
                'price_spread': price_spread,
                'final_energy_level': storage.energy_level,
                'npv': economic_results['npv']['npv'] if economic_results else None
            }
            
            results.append(result)
            
            # Konfiguration für diese Simulation speichern
            save_config_snapshot(window_dir, window_size)
            
            # Zusammenfassung für diese Window Size erstellen
            create_window_summary(window_dir, result, daily_results)
            
            # Ausführungszeit für diese Simulation
            elapsed_time = time.time() - start_time
            logger.info(f"Simulation für Window Size {window_size} abgeschlossen in {elapsed_time:.2f} Sekunden")
            
        except Exception as e:
            logger.error(f"Fehler bei Window Size {window_size}: {str(e)}")
            results.append({
                'window_size': window_size,
                'window_minutes': window_size * 15,
                'window_hours': window_size / 4,
                'profit': 0,
                'transactions': 0,
                'cycles': 0,
                'error': str(e)
            })
    
    # Ergebnisse in DataFrame konvertieren
    results_df = pd.DataFrame(results)
    
    # Ergebnisse speichern
    results_csv_path = os.path.join(base_output_dir, "window_size_results.csv")
    results_df.to_csv(results_csv_path, index=False)
    
    # Zusammenfassende Visualisierung der Ergebnisse
    create_results_visualization(results_df, base_output_dir)
    
    # Vergleichsdiagramme erstellen
    create_comparison_plots(results_df, base_output_dir)
    
    # Gesamtlaufzeit berechnen
    total_elapsed_time = time.time() - total_start_time
    hours, remainder = divmod(total_elapsed_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    logger.info(f"\nGesamtlaufzeit: {int(hours):02d}:{int(minutes):02d}:{seconds:.2f}")
    
    logger.info(f"Analyse abgeschlossen. Ergebnisse gespeichert in: {base_output_dir}")
    
    return results_df, base_output_dir


def save_config_snapshot(output_dir, window_size):
    """Speichert einen Snapshot der Konfiguration mit der aktuellen Window Size."""
    import inspect
    
    config_file_path = os.path.join(output_dir, "config_snapshot.py")
    
    with open(config_file_path, 'w', encoding='utf-8') as f:
        f.write("# Konfiguration zum Zeitpunkt der Simulation\n")
        f.write(f"# Window Size: {window_size}\n\n")
        
        for name, value in inspect.getmembers(config):
            if not name.startswith('__') and not inspect.isfunction(value) and not inspect.ismodule(value):
                if name == 'ROLLING_WINDOW_SIZE':
                    f.write(f"{name} = {window_size}  # Überschrieben für diese Simulation\n")
                elif isinstance(value, str):
                    f.write(f"{name} = '{value}'\n")
                else:
                    f.write(f"{name} = {value}\n")


def create_window_summary(output_dir, result, daily_results):
    """Erstellt eine Zusammenfassungsdatei für eine Window Size."""
    summary_path = os.path.join(output_dir, "summary.txt")
    
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write(f"ZUSAMMENFASSUNG FÜR WINDOW SIZE {result['window_size']}\n")
        f.write("="*60 + "\n\n")
        
        f.write("PARAMETER:\n")
        f.write(f"- Window Size: {result['window_size']} Intervalle ({result['window_minutes']} Minuten)\n")
        f.write(f"- Horizont: {result['window_hours']:.2f} Stunden\n\n")
        
        f.write("ERGEBNISSE:\n")
        f.write(f"- Jahresgewinn: {result['profit']:,.2f} €\n")
        f.write(f"- Anzahl Transaktionen: {result['transactions']}\n")
        f.write(f"- Batteriezyklen: {result['cycles']:.2f}\n")
        f.write(f"- Ø Kaufpreis: {result['avg_charge_price']:.2f} €/MWh\n")
        f.write(f"- Ø Verkaufspreis: {result['avg_discharge_price']:.2f} €/MWh\n")
        f.write(f"- Preisspreizung: {result['price_spread']:.2f} €/MWh\n")
        
        if result['cycles'] > 0:
            f.write(f"- Gewinn pro Zyklus: {result['profit']/result['cycles']:.2f} €/Zyklus\n")
        
        if result['npv'] is not None:
            f.write(f"- NPV: {result['npv']:,.2f} €\n")
        
        # Monatsstatistiken
        if daily_results:
            monthly_profits = {}
            for dr in daily_results:
                month = dr['date'].strftime('%Y-%m')
                if month not in monthly_profits:
                    monthly_profits[month] = 0
                monthly_profits[month] += dr['profit']
            
            f.write("\nMONATSGEWINNE:\n")
            for month, profit in sorted(monthly_profits.items()):
                f.write(f"- {month}: {profit:,.2f} €\n")


def create_comparison_plots(results_df, output_dir):
    """Erstellt zusätzliche Vergleichsdiagramme zwischen den Window Sizes."""
    
    # NPV Vergleich
    if 'npv' in results_df.columns and not results_df['npv'].isna().all():
        plt.figure(figsize=(12, 6))
        plt.plot(results_df['window_hours'], results_df['npv'], 'o-', linewidth=2, color='#D62728')
        plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        plt.grid(True, alpha=0.3)
        plt.title('NPV nach Window Size', fontsize=14, fontweight='bold')
        plt.xlabel('Horizont (Stunden)', fontsize=12)
        plt.ylabel('NPV (€)', fontsize=12)
        plt.savefig(os.path.join(output_dir, 'npv_comparison.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    # Effizienz-Vergleich: Gewinn pro Transaktion
    results_df['profit_per_transaction'] = np.where(
        results_df['transactions'] > 0,
        results_df['profit'] / results_df['transactions'],
        0
    )
    
    plt.figure(figsize=(12, 6))
    plt.plot(results_df['window_hours'], results_df['profit_per_transaction'], 'o-', 
             linewidth=2, color='#FF7F0E')
    plt.grid(True, alpha=0.3)
    plt.title('Gewinn pro Transaktion nach Window Size', fontsize=14, fontweight='bold')
    plt.xlabel('Horizont (Stunden)', fontsize=12)
    plt.ylabel('Gewinn pro Transaktion (€)', fontsize=12)
    plt.savefig(os.path.join(output_dir, 'profit_per_transaction.png'), dpi=300, bbox_inches='tight')
    plt.close()


def create_results_visualization(results_df, output_dir):
    """Erstellt die zusammenfassenden Visualisierungen mit verbesserter Tabellendarstellung."""
    logger = logging.getLogger(__name__)
    
    if results_df.empty:
        logger.warning("Keine Ergebnisse für Visualisierung vorhanden.")
        return
    
    try:
        # 1. Hauptvisualisierung: Profit, Transaktionen und Zyklen
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 12))
        
        # Profit vs. Window Size
        ax1.plot(results_df['window_hours'], results_df['profit'], 'o-', 
                 linewidth=2, markersize=8, color='#1F77B4')
        
        # Markiere den optimalen Punkt
        if not results_df['profit'].isna().all():
            optimal_idx = results_df['profit'].idxmax()
            optimal_window = results_df.loc[optimal_idx, 'window_hours']
            optimal_profit = results_df.loc[optimal_idx, 'profit']
            
            ax1.axvline(x=optimal_window, color='red', linestyle='--', 
                      label=f'Optimum: {optimal_window:.2f} h')
            ax1.scatter([optimal_window], [optimal_profit], color='red', s=100, zorder=5)
            
            ax1.annotate(f'Optimaler Horizont: {optimal_window:.2f} h\nGewinn: {optimal_profit:,.2f} €',
                       xy=(optimal_window, optimal_profit),
                       xytext=(optimal_window + 0.5, optimal_profit * 1.05),
                       arrowprops=dict(arrowstyle='->'))
        
        ax1.grid(True, alpha=0.3)
        ax1.set_title('Einfluss des Rolling-Window-Horizonts auf den Arbitragegewinn', 
                     fontsize=14, fontweight='bold')
        ax1.set_xlabel('Horizont (Stunden)', fontsize=12)
        ax1.set_ylabel('Arbitragegewinn (€)', fontsize=12)
        ax1.legend()
        
        # Transaktionen und Zyklen
        ax2.plot(results_df['window_hours'], results_df['transactions'], 'o-', 
                label='Anzahl Trades', color='#FF7F0E')
        ax2.set_ylabel('Anzahl Transaktionen', fontsize=12, color='#FF7F0E')
        ax2.tick_params(axis='y', labelcolor='#FF7F0E')
        
        ax2_twin = ax2.twinx()
        ax2_twin.plot(results_df['window_hours'], results_df['cycles'], 'o-', 
                     label='Zyklen', color='#2CA02C')
        ax2_twin.set_ylabel('Batteriezyklen', fontsize=12, color='#2CA02C')
        ax2_twin.tick_params(axis='y', labelcolor='#2CA02C')
        
        ax2.grid(True, alpha=0.3)
        ax2.set_xlabel('Horizont (Stunden)', fontsize=12)
        ax2.set_title('Handelsaktivität und Batteriezyklen', fontsize=14, fontweight='bold')
        
        # Preisspreizung
        ax3.plot(results_df['window_hours'], results_df['price_spread'], 'o-', 
                linewidth=2, color='#9467BD')
        ax3.grid(True, alpha=0.3)
        ax3.set_title('Erzielte Preisspreizung', fontsize=14, fontweight='bold')
        ax3.set_xlabel('Horizont (Stunden)', fontsize=12)
        ax3.set_ylabel('Durchschnittliche Preisspreizung (€/MWh)', fontsize=12)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, 'window_size_analysis_complete.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2. Gewinn pro Zyklus vs. Window Size
        plt.figure(figsize=(10, 6))
        
        # Berechne Gewinn pro Zyklus
        results_df['profit_per_cycle'] = np.where(
            results_df['cycles'] > 0,
            results_df['profit'] / results_df['cycles'],
            0
        )
        
        plt.plot(results_df['window_hours'], results_df['profit_per_cycle'], 'o-', 
                linewidth=2, color='#D62728')
        plt.grid(True, alpha=0.3)
        plt.title('Effizienz: Gewinn pro Batteriezyklus nach Horizont', 
                 fontsize=14, fontweight='bold')
        plt.xlabel('Horizont (Stunden)', fontsize=12)
        plt.ylabel('Gewinn pro Zyklus (€/Zyklus)', fontsize=12)
        plt.savefig(os.path.join(output_dir, 'profit_per_cycle_analysis.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # 3. Verbesserte Zusammenfassende Statistiktabelle
        fig, ax = plt.subplots(figsize=(14, 10))
        ax.axis('tight')
        ax.axis('off')
        
        # Tabellendaten vorbereiten
        table_data = []
        for _, row in results_df.iterrows():
            table_data.append([
                f"{row['window_size']}",
                f"{row['window_hours']:.2f}",
                f"{row['profit']:,.0f}",
                f"{row['transactions']}",
                f"{row['cycles']:.1f}",
                f"{row['price_spread']:.2f}",
                f"{row['profit_per_cycle']:.2f}" if row['cycles'] > 0 else "0.00"
            ])
        
        # Verbesserte Tabelle mit mehr Platz für Header
        table = ax.table(cellText=table_data,
                        colLabels=['Window\nSize', 'Horizont\n(Stunden)', 'Gewinn\n(€)', 
                                  'Trans-\naktionen', 'Zyklen', 'Preis-\nspreizung\n(€/MWh)', 
                                  'Gewinn/\nZyklus\n(€)'],
                        cellLoc='center',
                        loc='center')
        
        # Styling der Tabelle verbessern
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.8)
        
        # Header-Zeile höher machen für bessere Lesbarkeit
        for (row, col), cell in table.get_celld().items():
            if row == 0:  # Header-Zeile
                cell.set_height(0.15)
                cell.set_text_props(fontweight='bold')
                cell._text.set_verticalalignment('center')
        
        # Optimale Zeile hervorheben
        if not results_df['profit'].isna().all():
            optimal_idx = results_df['profit'].idxmax()
            for j in range(len(table_data[0])):
                table[(optimal_idx + 1, j)].set_facecolor('#FFE6E6')
        
        plt.title('Grid-Search-Ergebnisse für das Gesamtjahr 2024', fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.subplots_adjust(bottom=0.1)
        plt.savefig(os.path.join(output_dir, 'results_summary_table.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        logger.info("Zusammenfassende Visualisierungen erstellt.")
        
    except Exception as e:
        logger.error(f"Fehler bei der Visualisierung der Ergebnisse: {str(e)}")


# Wenn direkt ausgeführt
if __name__ == "__main__":
    results_df, output_dir = test_window_sizes_with_full_visualization()
    
    # Zeige die optimale Window Size an
    if not results_df.empty and not results_df['profit'].isna().all():
        optimal_idx = results_df['profit'].idxmax()
        optimal_window = results_df.loc[optimal_idx, 'window_size']
        optimal_profit = results_df.loc[optimal_idx, 'profit']
        optimal_npv = results_df.loc[optimal_idx, 'npv'] if 'npv' in results_df.columns else None
        
        print(f"\n{'='*80}")
        print(f"OPTIMALER ROLLING-WINDOW-HORIZONT: {optimal_window} (= {optimal_window*15} Minuten)")
        print(f"MAXIMALER ARBITRAGEGEWINN: {optimal_profit:,.2f} €")
        if optimal_npv is not None:
            print(f"NPV BEI OPTIMALEM HORIZONT: {optimal_npv:,.2f} €")
        print(f"{'='*80}")

2025-08-08 14:37:47,982 - INFO - Starte erweiterte Analyse verschiedener Window Sizes mit vollständigen Visualisierungen
2025-08-08 14:37:47,984 - INFO - Starte Einlesen der Datei: C:\Users\tillw\OneDrive\Bachelorarbeit\00_Großhandelspreise\SMARD_15min_Jahr 2024_DE\Gro_handelspreise_202401010000_202501010000_Viertelstunde.csv
2025-08-08 14:37:48,037 - INFO - Erfolgreich eingelesen: 35136 Zeilen
2025-08-08 14:37:48,177 - INFO - Zeitreihe konvertiert: 2024-01-01 00:00:00 bis 2024-12-31 23:45:00
2025-08-08 14:37:48,183 - INFO - 
Aufbereitete Daten:
2025-08-08 14:37:48,185 - INFO - Zeitraum: 01.01.2024 bis 31.12.2024
2025-08-08 14:37:48,186 - INFO - Anzahl der Datenpunkte: 35136
2025-08-08 14:37:48,187 - INFO - Preisbereich: -135.45 € bis 936.28 €/MWh
2025-08-08 14:37:48,188 - INFO - Durchschnittspreis: 78.51 €/MWh
2025-08-08 14:37:48,190 - INFO - Preisvolatilität (Stdabw.): 52.72 €/MWh
2025-08-08 14:37:48,202 - INFO - Anzahl der Tage im Datensatz: 366
2025-08-08 14:37:48,204 - INFO - 
202

Keine Transaktionen für Preisanalyse gefunden.


2025-08-08 14:38:29,406 - INFO - Fortschritt: 97.7% (34345/35136 Zeitpunkte)
2025-08-08 14:38:29,633 - INFO - Perzentil-basierte TH abgeschlossen. Energieverlauf mit 35136 Einträgen erstellt.
2025-08-08 14:38:29,634 - INFO - Optimierung abgeschlossen in 10.23 Sekunden
2025-08-08 14:38:29,635 - INFO - Gesamtzahl Transaktionen: 0
2025-08-08 14:38:29,635 - INFO - Gesamtgewinn: 0.00 €
2025-08-08 14:38:29,637 - INFO - Gesamtzyklen: 0.00
2025-08-08 14:38:29,637 - INFO - Gruppiere Transaktionen und Energiehistorie nach Datum...
2025-08-08 14:38:38,699 - INFO - Erstelle tägliche Ergebnisse...
2025-08-08 14:38:38,700 - INFO - Verarbeite Tag 1/366
2025-08-08 14:38:39,310 - INFO - Verarbeite Tag 51/366
2025-08-08 14:38:39,908 - INFO - Verarbeite Tag 101/366
2025-08-08 14:38:40,507 - INFO - Verarbeite Tag 151/366
2025-08-08 14:38:41,110 - INFO - Verarbeite Tag 201/366
2025-08-08 14:38:41,736 - INFO - Verarbeite Tag 251/366
2025-08-08 14:38:42,554 - INFO - Verarbeite Tag 301/366
2025-08-08 14:38:43

Keine Transaktionen für Preisanalyse gefunden.


2025-08-08 14:39:12,708 - INFO - Fortschritt: 58.1% (20400/35136 Zeitpunkte)
2025-08-08 14:39:19,470 - INFO - Perzentil-basierte TH abgeschlossen. Energieverlauf mit 35136 Einträgen erstellt.
2025-08-08 14:39:19,472 - INFO - Optimierung abgeschlossen in 16.77 Sekunden
2025-08-08 14:39:19,476 - INFO - Gesamtzahl Transaktionen: 2582
2025-08-08 14:39:19,478 - INFO - Gesamtgewinn: 13581.68 €
2025-08-08 14:39:19,480 - INFO - Gesamtzyklen: 161.25
2025-08-08 14:39:19,482 - INFO - Gruppiere Transaktionen und Energiehistorie nach Datum...
2025-08-08 14:39:34,786 - INFO - Erstelle tägliche Ergebnisse...
2025-08-08 14:39:34,788 - INFO - Verarbeite Tag 1/366
2025-08-08 14:39:35,867 - INFO - Verarbeite Tag 51/366
2025-08-08 14:39:36,972 - INFO - Verarbeite Tag 101/366
2025-08-08 14:39:38,085 - INFO - Verarbeite Tag 151/366
2025-08-08 14:39:39,195 - INFO - Verarbeite Tag 201/366
2025-08-08 14:39:40,333 - INFO - Verarbeite Tag 251/366
2025-08-08 14:39:41,702 - INFO - Verarbeite Tag 301/366
2025-08-08